<a href="https://colab.research.google.com/github/LeoFernanndes/notebooks/blob/master/housing_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [0]:
import pandas as pd


url = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_train.csv'
df1 = pd.read_csv(url)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [0]:
housing = df1.copy()

# starting by filling the blank spaces 
for column in housing.columns:
    if housing[column].dtype != 'object':
        housing[column].fillna(housing[column].mean(), inplace= True)
    else:
        housing[column] = housing[column].astype('str')
        housing[column].fillna(housing[column].mode(), inplace= True)

In [84]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


features = ['OverallQual', 'GrLivArea', 'TotalBsmtSF', 'BsmtFinSF1', '2ndFlrSF',
       'GarageArea', '1stFlrSF', 'GarageCars', 'LotArea', 'YearBuilt',
       'MasVnrArea', 'TotRmsAbvGrd', 'YearRemodAdd', 'BsmtQual', 'FullBath',
       'Neighborhood', 'LotFrontage', 'BsmtUnfSF', 'GarageYrBlt',
       'GarageFinish', 'OverallCond', 'CentralAir', 'OpenPorchSF', 'Id',
       'ExterQual', 'WoodDeckSF', 'GarageType', 'KitchenQual', 'FireplaceQu',
       'MoSold', 'LotShape', 'BsmtFinType1', 'Fireplaces', 'Exterior2nd',
       'BsmtExposure', 'MSSubClass', 'SaleCondition', 'MSZoning', 'YrSold',
       'BsmtFullBath', 'Exterior1st', 'BedroomAbvGr', 'HalfBath', 'MasVnrType',
       'LandContour', 'LotConfig', 'PoolArea', 'EnclosedPorch', 'Condition1',
       'HouseStyle', 'ScreenPorch', 'HeatingQC', 'ExterCond', 'RoofStyle',
       'Alley', 'BsmtFinSF2', 'Foundation', 'SaleType', 'BsmtFinType2',
       'Functional', 'KitchenAbvGr', 'Fence', 'BsmtHalfBath', 'LandSlope',
       'PoolQC', 'BsmtCond', 'RoofMatl', 'PavedDrive', 'BldgType', '3SsnPorch',
       'GarageCond', 'LowQualFinSF', 'Electrical', 'GarageQual', 'MiscVal',
       'Heating', 'Condition2', 'MiscFeature', 'Street', 'Utilities']



x_opt = housing[features]
y_opt = housing['SalePrice']


le_opt = LabelEncoder()
for column in x_opt.columns:
    if x_opt[column].dtype == 'object':
        x_opt[column] = le_opt.fit_transform(x_opt[column])

df_ = x_opt
df_['salesprice'] = y_opt

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [85]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor


n = 5

df = df_[features[0:n+1]].iloc[:1200,:]

m = 10
erro1 = 0


for i in(range(m)):
  
  x_train, x_test, y_train, y_test = train_test_split(df[df.columns[:-1]], df[df.columns[-1]], test_size= 0.2, random_state= m**2)

     
  reg = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
  
  reg = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
              max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
              oob_score=False, random_state=None, verbose=0, warm_start=False)
  

  reg.fit(x_train, y_train)
  y_pred = reg.predict(x_test)

  erro = np.sqrt(mean_squared_log_error(y_test, y_pred))
  print(erro)
  erro1 += erro

erro1/m


1.3507109232006527
1.3488063461955602
1.352449748804496
1.3448134963633156
1.3489334901479708
1.3455139097457538
1.3474376433875652
1.3489414530511215
1.3470614062783206
1.3488696023005566


1.3483538019475314

In [86]:
x_prova_real = df_[features[0:n+1]].iloc[1200:,:-1]
y_prova_real = df_['salesprice'].iloc[1200:]

y_pred_prova_real = reg.predict(x_prova_real)

erro_prova_real = np.sqrt(mean_squared_log_error(y_prova_real, y_pred_prova_real))
erro_prova_real



5.929636827153551

In [87]:
df[df.columns[:-1]]

,OverallQual,GrLivArea,TotalBsmtSF,BsmtFinSF1,2ndFlrSF
0,7,1710,856,706,854
1,6,1262,1262,978,0
2,7,1786,920,486,866
3,7,1717,756,216,756
4,8,2198,1145,655,1053
...,...,...,...,...,...
1195,6,1456,728,0,728
1196,7,1863,879,0,984
1197,7,1690,815,0,875
1198,7,1212,1212,0,0


In [0]:
# making the predictions over submission set
# loading submission data
url_data = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_test.csv'
data = pd.read_csv(url_data)

In [0]:
# filling in the gaps in the same manner as used in the trein set
for column in data.columns:
    if data[column].dtype != 'object':
        data[column].fillna(data[column].mean(), inplace= True)
    else:
        data[column] = data[column].astype('str')
        data[column].fillna(data[column].mode(), inplace= True)

# using label encoder over the test set the same way as used in trein set 
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = le_opt.fit_transform(data[column])

In [0]:
# df for kaggle evaluation
final = reg.predict(data[df.columns[:-1]])
envio = pd.DataFrame({'Id': data['Id'], 'SalePrice': final})

In [0]:

# saving the df in a csv to be uploaded

#path = r'C:\Users\Avell\Desktop\Python\github\datasets'
#nome = '\housing_submission_2.csv'
#envio.to_csv(path+nome, index= False)


In [91]:
 reg = XGBRegressor()
 reg.fit(x_prova_real, y_prova_real)

[01:40:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)